In [1]:
class Node:
    def __init__(self, value=None, next=None, prev=None):
        self.value = value
        self.next = next
        self.prev = prev
    def __str__(self):
        return f"<Node: value: {self.value}>"
    # 显示属性
    __repr__ = __str__

class DBL:
    def __init__(self):
        node = Node("root")
        node.prev = node
        node.next = node
        self.root = node
        self.lens = 0
    def iter_item(self):
        node = self.root
        while node.next != self.root:
            node = node.next
            yield node
    @property
    def head(self):
        return self.root.next
    @property
    def tail(self):
        return self.root.prev
    
    def append(self, value):
        node = Node(value)
        self.tail.next = node
        node.prev = self.tail
        node.next = self.root
        self.root.prev = node
        self.lens += 1
    
    def remove(self, node):
        if node == self.root:
            return False
        node.next.prev = node.prev
        node.prev.next = node.next
        del node
        self.lens -= 1
        return True
    
    def __str__(self):
        return "->".join([str(node.value) for node in self.iter_item()])

    __repr__ = __str__
    
class LRU:
    '''
    使用双向链表、哈希表实现
    dict[key] = node
    node存着value
    哈希表用来实现对key的快速查找,因为限定容量，因此如果存在，查找复杂度O(1)
    双向链表来管理使用的先后顺序，当需要删除时，剔除最近没有被用到的node
    为什么要用双向链表？
    所需的操作有：向链表尾部插入node，删除头节点的node，将任意位置的node移动到尾部
    '''
    def __init__(self, size = 10):
        self.size = size
        self._link = DBL()
        self._cache = dict()
        
    def _move_to_recent(self, node):
        if node == self._link.tail:
            return
        #从原来的位置删除
        node.prev.next = node.next
        node.next.prev = node.prev
        #添加到尾部
        now_tail = self._link.tail
        now_tail.next = node
        node.prev = now_tail
        node.next = self._link.root
        self._link.root.prev = node
    
    def _append(self, k, v):
        self._link.append(v)
        self._cache[k] = self._link.tail
        #让节点知道Key是什么 感觉也可以不加
        #self._link.tail.cache_key = k
    
    def _expired_not_used(self):
        need_expired = self._link.head
        self._link.remove(need_expired)
    
    def get(self, k):
        node = self._cache.get(k)
        if not node:
            return
        self._move_to_recent(node)
        return node.value
    
    def put(self, k, v):
        node = self._cache.get(k)
        if node:
            node.value = v
            self._move_to_recent(node)
        else:
            if self._link.lens == self.size:
                self._expired_not_used()
            self._append(k, v)
    def __str__(self):
        return "->".join([f"{node.value}" for node in self._link.iter_item()])
    __repr__ = __str__